# LMC Analysis: Part II
## Mock Observables from Models of the Large Magellanic Cloud
### Author: Chad Bustard

This notebook goes through a few analyses of simulation data, specifically the simulations published in Bustard et al. 2020 of ram pressure stripping and supernova-driven outflows from the Large Magellanic Cloud. 

#### Main publication: https://ui.adsabs.harvard.edu/abs/2020ApJ...893...29B/abstract

In part II (this notebook), we will plot values of "ram pressure" across the LMC disk and tabulate them at specific points where there are observational sightlines (this part, followed by an analysis of correlations between local ram pressure and observed ion abundances forms the basis for a funded Hubble Space Telescope (HST) grant (PI: Yong Zheng, Co-I: Chad Bustard)

## Ram Pressure
The ram pressure acting on an object moving with velocity $\bf{v_{infall}}$ through a static background gas with density $\rho$ is, in most basic form: 

(most basic) $P_{\rm ram} = \rho v_{\rm infall}^{2}$

This pressure counteracts the gravitational "tension" force of the galaxy to strip gas from the galaxy.

### Towards the *local* ram pressure in different regions of the LMC
The above equation would describe the interaction between a galaxy and a headwind *if the velocity offset between the galaxy and headwind were constant across the disk*; however, to understand how ram pressure affects local star formation rates, local ion abundances, etc. we would like a more *local* description of ram pressure.

To a first approximation, we can assume the LMC is a rotating disk infalling at an arbitrary inclination. We have to account for the fact that, from the perspective of the LMC, some parts of the disk are rotating *into* the headwind while others are rotating *away* from the headwind. Therefore, the velocity vector of interest is really $v_{\rm infall} - v_{\rm rot}$, where $v_{\rm rot}(x_{\rm LMC}, y_{\rm LMC}) = (v_{\rm rot,x}, v_{\rm rot,y})$ is the rotation vector (in coordinates such that $\hat{z}$ is along the axis of disk rotation, and $v_{\rm infall} = (v_{x}, v_{y}, v_{z})$ is the infall velocity vector of the LMC "as a whole" motivated by observations and more complete simulations of the LMC's interaction (including it's complicated gravitational dance with it's neighboring SMC): 

(including rotation) ${\bf P}_{\rm ram}(x_{LMC}, y_{LMC}) = \rho \left((v_{x}-v_{rot,x})^{2}\hat{x} + (v_{y}-v_{rot,y})^{2}\hat{y} + v_{z}^{2}\hat{z}\right)$. 

Note that we've written the ram pressure as a vector and with dependence on LMC-centered coordinates $(x_{LMC}, y_{LMC})$ in the plane perpendicular to the disk axis of rotation. So to know the ram pressure impacting any point $(x_{LMC}, y_{LMC})$ across the disk, we need the following ingredients:

    1. The density of the Milky Way halo at the current LMC location. 
    
    2. The infall velocity vector -- really, a combination of the LMC's infall velocity and a sense of it's inclination during infall (you can imagine a disk "slicing" through the air vs a disk "pancaking" through the air).
    
    3. The rotation vector
    
In our paper, we more fully account for how, over the course of its infall, the LMC changes both its distance from the Milky Way (and therefore the increasing halo gas density it interacts with as it gets closer and closer to the Milky Way) and its inclination. 

It's not worth going into all the observational measurements that lead to estimates of the halo gas density, the infall velocity, etc. Just know that $\rho$, $v_{x}$, $v_{y}$, $v_{z}$ are well-motivated but, of course, have some uncertainties.

The rotation velocity can be calculated by assuming hydrostatic balance between the LMC's centrifugal force and gravitational force...that's how it is done in the Fortran-based FLASH simulations I created to model the LMC...but in the next cell, we'll just take the rotation velocity as a function of $r_{LMC}$ from Salem et al. 2015. 

An additional complexity we can account for is that galaxies are constantly expelling and accreting gas through the *disk-halo interface*, here the region between the LMC's disk and its surrounding *circumgalactic medium.* These gas flows are "very" challenging to model correctly (no model is ever correct, some are just closer to reality), but my own simulations (Bustard et al. 2020) are a first-of-their-kind attempt to model the supernova-driven galactic outflows from the LMC, connecting them to the temporally and spatially resolved star formation history that ultimately determines the time and locations of galactic outflow driving. Check out the .F90 file in this directory for the initial condition and boundary condition code (the FLASH code this is built on is proprietary, so that's all I can post on github).

Generalizing $v_{\rm rot}$ to $v_{\rm local} = (v_{loc,x},v_{loc,y},v_{loc,z})$, which includes all velocity information relative to the frame of rest of the LMC, the full ram pressure can be estimated as 

(including local gas flows) ${\bf P}_{\rm ram}(x_{LMC}, y_{LMC}, z_{LMC}) = \rho \left((v_{x}-v_{loc,x})^{2}\hat{x} + (v_{y}-v_{loc,y})^{2}\hat{y} + (v_{z} - v_{loc,z})^{2}\hat{z}\right)$

Note that, while rotation doesn't alter ram pressure in the $\hat{z}$ direction, other gas flows surely can (and do) and therefore are included here. 

In [ ]:
# Import all the packages we need
import yt
import trident
from trident import LightRay
import aplpy
import numpy as np
#import matplotlib as plt
from yt.units.yt_array import YTQuantity
from yt import YTArray
import h5py
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy import wcs
from astropy.wcs import WCS
import csv
import pandas as pd

!pip install cmasher # a nice repository of colorblind-friendly, aesthetically pleasing colormaps
import cmasher as cmr

In [ ]:
# Get the HDF5 file from my Google Drive
!pip install gdown
!gdown "https://drive.google.com/uc?id=1BCvIdYOA3VZj91Fdh9seOki0ww3Rqiky"

In [ ]:
# Rotation velocity from Salem+ 2015
# In the form I obtained it, the velocity is tabulated as a function of cylindrical radius (zero radius is the center of the LMC)

# Let's use scipy to fit an exponential curve to our data 
from scipy.optimize import curve_fit
 
def func(x,a,b,c):
    return a * np.exp(-b * x) + c

# radius in units of kiloparsecs (kpc)
r_data = [0.025,0.091,0.223,0.364,0.496,0.709,0.84,1.043,1.316,1.569,1.893,2.227,2.642,3.107,3.573,4.038,4.464,4.879,5.294,5.698,6.144,6.579,6.953,7.308,7.763,8.381,9.008,9.524, 9.975]

# rotational velocity
v_data = [21.671,25.196,30.809,35.64,40.47,45.822,49.086,53.264,58.355,62.533,66.449,69.582,72.715,75.718,77.546,79.373,80.287,81.07,81.593,81.984,82.115,82.245,82.245,81.984,  82.115,81.854,81.723,81.07,80.94]


r_arr = np.arange(0.01,10,0.01)
popt, pcov = curve_fit(func, r_data, v_data)
#print(popt)
plt.plot(r_data,v_data,label="From Salem2015")
plt.plot(r_arr, func(r_arr, *popt), 'r-', label='fit: a=%5.3f, b=%5.3f, c=%5.3f' % tuple(popt))
plt.xlabel("x (kpc)")
plt.ylabel("Velocity (km/s)")
plt.legend()
plt.show()


This curve with appropriate values for a, b, c fits the data extremely well (that functional form is physically motivated, so I knew it would work well), so let's take the magnitude of the velocity to be $v(r) = a e^{-br} + c$ from now on and proceed to calculate the ram pressure.

For the remainder of this notebook, we employ the `yt` package, a visualization and analysis toolkit for astrophysics: https://yt-project.org/. We first create a new function for the ram pressure given $v_{\rm infall}$ and $v_{\rm rot}$

In [ ]:
# define new functions for ram pressure
def _ramPres_rot(field, data): # ram pressure #2, including rotation effect but not local gas flows
        rad = np.sqrt((data['x']**2.0 + data['y']**2.0)) # radius from center of LMC
        velrot_y = 1.e5*func(rad,*popt)*data['x']/rad*YTQuantity(1,'cm/s') # v_y in units of km/s
        velrot_x = 1.e5*func(rad,*popt)*data['y']/rad*YTQuantity(1,'cm/s') # v_x
        
        # density of the Milky Way halo at present-day position of LMC is rho = 1.65e-28 g/cm^3
        # v_infall = (220, 159, 192) km/s
        val = 1.65e-28*YTQuantity(1,'g/cm**3')*((220.0*1e5*data['ones']*YTQuantity(1,'cm/s') - velrot_x)**2.0 + (159.0*1e5*data['ones']*YTQuantity(1,'cm/s') - velrot_y)**2.0 + (192.0*1e5*data['ones']*YTQuantity(1,'cm/s'))**2.0)
        return val

yt.add_field(("gas","ramPres_rot"), function=_ramPres_rot,units="dyne/cm**2")
 
 

In [ ]:
# Restrict our plot to just the galaxy (cells with an ISM tag > 0.6)
cut_data = dd.cut_region('(obj["ism "] > 0.6)')

prj_fits = yt.FITSOffAxisProjection(ds,L,'ramPres_rot',center = (0,0,0),width=(20, 'kpc'), weight_field="ones",data_source=cut_data)
sky_center = [79.0,-68.68] # in degrees
sky_scale = (4123.71, "arcsec/kpc") # could also use a YTQuantity
prj_fits.create_sky_wcs(sky_center, sky_scale, ctype=["RA---TAN","DEC--TAN"], replace_old_wcs=True)
prj_fits.writeto("LMC_ram_RP_rotate.fits", clobber=True)

In [ ]:
image_data = fits.getdata('LMC_ram_RP_rotate.fits')
print(type(image_data))
print(image_data.shape)

fig = aplpy.FITSFigure("LMC_ram_RP_rotate.fits")
fig.add_grid()
fig.show_colorscale(cmap="kamae_r",stretch = 'linear', vmin=1E-13, vmax = 3.5E-13)
fig.tick_labels.set_xposition("top")
fig.ticks.set_xspacing(8.0)
fig.ticks.set_yspacing(8.0)
fig.add_colorbar()
 
# fig.tick_labels.set_xformat('hh:mm')
fig.tick_labels.set_xformat('dd')
fig.tick_labels.set_yformat('dd')
# fig.set_theme('pretty')
fig.colorbar.show(log_format=False)
fig.colorbar.set_font(size=20)
fig.show()

In [ ]:
# Read in table of (RA, DEC) values for sightlines through the LMC
inputArr = np.genfromtxt('ullyses_lmc_dr5_chadedits.csv',delimiter=',',invalid_raise=False)
ra_Yong = inputArr[:,0]
ra_Yong = ra_Yong[1:len(ra_Yong)]
dec_Yong = inputArr[:,1]
dec_Yong = dec_Yong[1:len(dec_Yong)]

In [ ]:
f = fits.open("LMC_ram_RP_rotate.fits")
w = wcs.WCS(f[0].header)
print(w)

In [ ]:
arr = f["ramPres_rot"].data

ra_test, dec_test = w.wcs_world2pix(ra_Yong, dec_Yong, 1)
plt.plot(ra_Yong,ra_test)
plt.xlabel("RA Inputs")
plt.ylabel("Pixel Outputs")
plt.show()

plt.plot(dec_Yong,dec_test)
plt.xlabel("DEC Inputs")
plt.ylabel("Pixel Outputs")
plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
ax = plt.subplot(projection=w)
plt.imsave('test.png',arr, origin='lower', cmap='kamae_r')
plt.xlabel(r'RA')
plt.ylabel(r'Dec')
 
overlay = ax.get_coords_overlay('icrs')
overlay.grid(color='white', ls='dotted')
 
RP_YongCoords = []
for j in range(0,len(ra_Yong)):
    ra, dec = w.wcs_world2pix(ra_Yong[j], dec_Yong[j], 1)
    # ra, dec = w.world_to_pixel(ra_Yong[j], dec_Yong[j]) #, 1))
    ra = int(ra)
    dec = int(dec)
    print("Celestial Coordinate inputs: ")
    print(ra_Yong[j])
    print(dec_Yong[j])
    print("Pixel Coordinate outputs: ")
    print(ra)
    print(dec)
    print("RP Value: ")
    # print(arr[ra,512-dec])
    # RP_YongCoords.append(arr[ra,512-dec])
    print(arr[ra,dec])
    RP_YongCoords.append(arr[dec,ra])

In [ ]:
RA, Dec = w.wcs_world2pix(ra_Yong, dec_Yong, 1)
newRA, newDec = w.wcs_pix2world(RA, Dec, 1)

plt.plot(ra_Yong,RA,'ko')
plt.show()
 
plt.plot(dec_Yong,Dec,'ko')
plt.show()

df = pd.read_csv('ullyses_lmc_dr5_chadedits.csv')
df['RP(dyne/cm^2)'] = RP_YongCoords
print(df)
 
sc = plt.scatter(ra_Yong,dec_Yong,c = RP_YongCoords,cmap="Greens")
plt.colorbar(sc)
plt.title("Ram Pressure (Rotation Included)")
plt.gca().invert_xaxis()
plt.xlabel("RA (ICRS)",fontsize=16)
plt.ylabel("DEC (ICRS)",fontsize=16)
plt.tight_layout()
plt.show()
 
df.to_csv('RP_Rotation_dr5_Sept2022.csv')

# Quick analysis of ram pressure vs star formation rate
A question of interest: does ram pressure correlate with star formation rate? We hypothesize that higher ram pressure causes more gas compression, leading to gravitational instability and gas collapse, eventually forming stars. With a birds-eye view of star formation in the LMC, combined with ram pressure values taken from my simulations, do we find a correlation?

In [ ]:
inputArr = np.genfromtxt('aligned_LMC_SFR_RP_rotation_ChadVersion.csv',delimiter=',',            invalid_raise=False)
SFR = inputArr[:,0]
SFR = SFR[1:len(SFR)]
ra = inputArr[:,1]
ra = ra[1:len(ra)]
dec = inputArr[:,2]
dec = dec[1:len(dec)]
RP = inputArr[:,3]
RP = RP[1:len(RP)]

plt.plot(RP, SFR,'bo')
plt.title("RP vs SFR (rotation included)")
plt.xlabel("Ram Pressure",fontsize=16)
plt.ylabel("SFR",fontsize=16)
plt.tight_layout()
plt.savefig("SFR_RP_wholeDisk.pdf")
plt.close()

Across *all* sightlines sampling the entire LMC disk, there's only a slight correlation. What if we cut the LMC into different quadrants?

But the sampling is clearly not spatially uniform...we're really interested in what the mean star formation rate is in different regions, as well as the variance in those regions. To that end, we can coarse-grain our data and look for correlations.

In [ ]:
# dec range is -65 to -72
# ra range is 70 to 90

numbin = 4
raGrid = np.arange(70,90,(20/numbin))
decGrid = np.arange(-65,-72,-(7/numbin))

print("Testing SFR: ")
test = np.where((dec > -70) & (dec < -64) & (ra < 90) & (ra > 80))
print(SFR[test])


#rows, cols = (len(raGrid), len(decGrid))
#SFR_new = [[0 for i in range(cols)] for j in range(rows)]
SFR_new = [0 for k in range(len(raGrid)*len(decGrid))]
RP_new = [0 for k in range(len(raGrid)*len(decGrid))]
counter = 0
for i in range(0,len(raGrid)-1):
    for j in range(0,len(decGrid)-1):
        counter +=1
        with warnings.catch_warnings():
            warnings.simplefilter("ignore", category=RuntimeWarning)
            location = np.where((dec < decGrid[j]) & (dec > decGrid[j+1]) & (ra < raGrid[i+1]) &   (ra > raGrid[i]))
            avgSFR = np.mean(10**(SFR[location]))
            avgRP = np.mean(RP[location])
            if (avgSFR > 0):
                SFR_new[counter] = np.log10(avgSFR)
                RP_new[counter] = np.log10(avgRP)
                
print("SFR averages: ")
print(SFR_new)

xnew = raGrid + (20/numbin)/2
ynew = decGrid -(7/numbin)/2

print(xnew)
print(ynew)

x,y = np.meshgrid(xnew, ynew)

plt.scatter(x,y, c = SFR_new,cmap='Blues_r',vmin = -8.0, vmax = -6)
plt.gca().invert_xaxis()
plt.xlabel("RA")
plt.ylabel("DEC")
plt.colorbar()
plt.title(r"Average SFR")
plt.savefig("AverageSFR_coarseRes.pdf")
plt.close()